In [2]:
import torch
torch_device = "cuda" if torch.cuda.is_available() else "cpu"
from tqdm.auto import tqdm
from torch import autocast
from PIL import Image
import cv2
import numpy as np
import math
import random
import bisect
import operator
import matplotlib.pyplot as plt
import copy
import sys
import os
import shutil

In [3]:
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler

# 1. Load the autoencoder model which will be used to decode the latents into image space. 
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

# 2. Load the tokenizer and text encoder to tokenize and encode the text. 
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# 3. The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.15.mlp.fc1.bias', 'vision_model.encoder.layers.12.mlp.fc1.weight', 'vision_model.encoder.layers.10.self_attn.v_proj.weight', 'vision_model.encoder.layers.19.self_attn.k_proj.weight', 'vision_model.encoder.layers.16.layer_norm1.weight', 'vision_model.encoder.layers.6.mlp.fc2.bias', 'vision_model.encoder.layers.10.mlp.fc2.bias', 'vision_model.encoder.layers.7.self_attn.v_proj.weight', 'vision_model.encoder.layers.5.layer_norm2.weight', 'vision_model.encoder.layers.17.self_attn.v_proj.bias', 'vision_model.encoder.layers.8.mlp.fc1.bias', 'vision_mode

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


In [10]:
from diffusers import LMSDiscreteScheduler

scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)



In [4]:
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device)

In [11]:
def func(latents,text_embeddings):
    
    num_inference_steps = 50            # Number of denoising steps

    guidance_scale = 7.5                # Scale for classifier-free guidance

    #############################################

    max_length = text_input.input_ids.shape[-1]
    uncond_input = tokenizer(
        [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
    )
    with torch.no_grad():
      uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]

    ###############################################

    embeddings = torch.cat([uncond_embeddings, text_embeddings])

    #############################################

    scheduler.set_timesteps(num_inference_steps)

    ##########################################

    latents_1 = latents * scheduler.init_noise_sigma

    #############################################



    for t in tqdm(scheduler.timesteps):
      # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
      latent_model_input_a = torch.cat([latents_1] * 2)

      latent_model_input = scheduler.scale_model_input(latent_model_input_a, t)

      # predict the noise residual
      with torch.no_grad():
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=embeddings).sample

      # perform guidance
      noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
      noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

      # compute the previous noisy sample x_t -> x_t-1
      latents_1 = scheduler.step(noise_pred, t, latents_1).prev_sample

    ###############################################

    # scale and decode the image latents with vae
    latents_3 = 1 / 0.18215 * latents_1

    with torch.no_grad():
      image = vae.decode(latents_3).sample


    ##########################################

    image_1 = (image / 2 + 0.5).clamp(0, 1)
    image_2 = image_1.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image_2 * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    result_image = pil_images[0]
    
    return result_image

In [43]:
#gaの関数
population = 3 #個体数
generations = 10 #世代数
X = 1
Y = 3
mutation = 1 #突然変異の個体数
# population = 3 #個体数
# generations = 2 #世代数
# X = 1
# Y = 3
# mutation = 1 #突然変異の個体数

if X*Y!=population:
    print('Error: parameter error', file=sys.stderr)
    sys.exit(1)
    
gene_length = 64
elite = 1 #エリートの数
initializa_txt_num = 100 #初期化個体においてtxtのベクトルをどれくらい元から変異させるか
batch_size = 1
height = 512                        # default height of Stable Diffusion
width = 512                        # default width of Stable Diffusion

if elite+mutation>population:
    print('Error: parameter error', file=sys.stderr)
    sys.exit(1)
    
def pil2cv(image):
    ''' PIL型 -> OpenCV型 '''
    new_image = np.array(image, dtype=np.uint8)
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif new_image.shape[2] == 4:  # 透過
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    return new_image


#最大42 大きければ大きいほどよい
# def calc_fitness(target,images):
#     fitness = []
#     for image in images:
#         target_hist = cv2.calcHist([target], [0], None, [256], [0, 256])
#         comparing_hist = cv2.calcHist([pil2cv(image)], [0], None, [256], [0, 256])
#         ret = cv2.compareHist(target_hist, comparing_hist, 0)
#         ans = 25*(np.exp(max(0,ret)))-24 #適合度の式
#         fitness.append(int(ans))
#     return fitness

#seed値を複数個用意
def initialize_gene(text_embeddings):
    arr = []
    for i in range(population):
        embeddings = text_embeddings.clone()
        seed_here = random.randrange(1000)
        latents_torch = torch.randn(
          (batch_size, unet.in_channels, height // 8, width // 8),
          generator=torch.manual_seed(seed_here),
        )
        latents = latents_torch.to(torch_device)
        for j in range(initializa_txt_num):
            a = random.randrange(77)
            b = random.randrange(768)
            text_embeddings[0][a][b] = np.random.randn()
        arr.append([latents,embeddings])    
    return arr


def save_image(diffusion_images,epoch):
    os.mkdir('./iec/epoch'+str(epoch))
    for i in range (len(diffusion_images)):
        diffusion_images[i].save("./iec/epoch"+str(epoch)+"/"+str(i)+".png")
    
    
def evolve(selected,genes):
    global evolve_explanation
    new_genes = []
    
    #エリート戦略 #選ばれた画像は次世代に残す(?)
    for i in range(population):
        if selected[i]==1:
            adding_gene = copy.deepcopy(genes[i])
            evolve_explanation += str(len(new_genes))+". elite"+str(i)+"\n"
            new_genes.append(adding_gene)
    
    #mutation, cross overで使う配列の準備
    index = []
    num = 0
    for i in range(population):
        num += 1+selected[i]*adapt
        index.append(num)
    index.append(num)
    
    #mutation
    for i in range(mutation):
        p1 = random.randrange(num)
        g1 = bisect.bisect(index,p1)
        evolve_explanation += str(len(new_genes))+". mutation "+str(g1)+"\n"
        new_gene1 = copy.deepcopy(genes[g1])
        for j in range(4):
            for k in range(64):
                for l in range(64):
                    mutation_flag = random.randrange(100)
                    if mutation_flag==0:
                        new_gene1[0][0][j][k][l] = np.random.randn()
        for j in range(77):
            for k in range(768):
                mutation_flag = random.randrange(100)
                if mutation_flag==0:
                    new_gene1[1][0][j][k] = np.random.randn()
        new_genes.append(new_gene1)

    # cross over
    for i in range(population-len(new_genes)):
        p1 = random.randrange(num)
        p2 = random.randrange(num)
        g1 = bisect.bisect(index,p1)
        g2 = bisect.bisect(index,p2)
        new_gene = copy.deepcopy(genes[g1])
        new_gene1 = copy.deepcopy(genes[g2])
        evolve_explanation += str(len(new_genes))+". crossover "+str(g1)+" and "+str(g2)+"\n"
        cross = random.randrange(gene_length)
        for j in range(4):
            for k in range(64):
                for l in range(64):
                    a = random.randrange(2)
                    if a==0:
                        new_gene[0][0][j][k][l] = new_gene1[0][0][j][k][l]
        for j in range(77):
            for k in range(768):
                new_gene[1][0][j][k] = new_gene1[1][0][j][k]
        new_genes.append(new_gene)
                    
    if len(new_genes)!=population:
        print('Error: evolve error', file=sys.stderr)
        sys.exit(1)
    return new_genes

In [44]:
##########################################################
#このセル実行前にiecディレクトリ用意&iec/epochディレクトリ削除
##########################################################
# tkinterの参考にしたサイト
# https://teratail.com/questions/202187
# buttonに関するサイト
# https://torimakujoukyou.com/python-tkinter-for-button/
# https://qiita.com/igor-bond16/items/39c8b75d844f30cea197
# 画像の切り替え
# https://joytas.net/programming/python/tkinter-img
# 画面の切り替え
# https://office54.net/python/tkinter/screen-change-tkraise
##########################################################

#ライブラリのインポート
import tkinter as tk
import random
import PIL
from PIL import Image,ImageTk
##############################################
#parameter
# X = 1
# Y = 3
image_padding = 300
imgs = [] # Imageを保持するリストを追加
buttons = []
selected = []
canvases = []
labels = []
for i in range(X*Y):
    selected.append(0)
epoch = 0 #現時点での世代数
adapt = 5 #iecにおいてuserに選ばれた遺伝子は選ばれていない遺伝子よりadapt倍選ばれやすい
evolve_explanation = ""
#diffusionの処理
target_image = Image.open("mountain.png").convert('RGB').resize((512,512))
target = pil2cv(target_image)
prompt = ["a photograph of a mountain range with a lot of greenery with roads along the foothills and large clouds in the sky during dusk"]
text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
with torch.no_grad():
  text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
genes = initialize_gene(text_embeddings)
##############################################
# web表示用の関数
# 選択されたときにボタンの色を変えて、選ばれた画像を記録
def click_button(item,btn):
    def nothing():
        if selected[item]==1:
            btn.config(bg='#E6E6E6')
            selected[item] = 0
        else:
            btn.config(bg='yellow')
            selected[item] = 1
    return nothing
# okボタンが押されたときに進化させて新しい画像を表示したい
def click_ok_button():
    global epoch
    #################
    # stable diffusionのコード
    #進化+選択
    #選択
    with open('selected.txt', 'a') as f:
        print(selected, file=f)
    #進化
    global genes
    genes = evolve(selected,genes)
    #画像生成
    diffusion_images.clear()
    for j in range(population):
        diffusion_image = func(genes[j][0],genes[j][1])
        diffusion_images.append(diffusion_image)
    #表示
    epoch += 1
    save_image(diffusion_images,epoch)
    ###################
    #新しい画像を表示
    for i in range(X*Y):
        img = Image.open('iec/epoch'+str(epoch)+'/' + str(i) + '.png')
        img = img.resize((256,256),Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        imgs.append(img) # Imageをリストに追加
        canvases[i].delete()
        canvases[i].create_image(3, 3, image=img, anchor=tk.NW,tag=str(i))
        #ボタンを未クリックにする
        buttons[i].config(bg='#E6E6E6')
        selected[i] = 0
    labels[0].config(text=evolve_explanation)
##############################################

if __name__ == "__main__":
    with autocast("cuda"):
        #ディレクトリ初期化
        if (os.path.exists('./iec')):
            shutil.rmtree('./iec/')
            os.mkdir('./iec')

        #初期配置
        diffusion_images = []
        for i in range(population):
            diffusion_image = func(genes[i][0],genes[i][1])
            diffusion_images.append(diffusion_image)
        save_image(diffusion_images,0)
        ############################################################################
        #webの初期配置
        root = tk.Tk()
        root.title("iec system")
        root.geometry("1500x900")
        #target image
        img = Image.open('mountain.png')
        img = img.resize((256,256),Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        imgs.append(img) # Imageをリストに追加
        canvas = tk.Canvas(bg = "purple", width=256, height=256)
        canvas.place(x=150, y=100) 
        canvas.create_image(3, 3, image=img, anchor=tk.NW,tag=str(i*Y+j))
        target_label = tk.Label(text="target image", background='orange', foreground='white',font=("Lucida Console","15"))
        target_label.place(x=150, y=70)
        for i in range(X):
            for j in range(Y):
                # 画像の設置
                img = Image.open('iec/epoch0/' + str(i*Y+j) + '.png')
                img = img.resize((256,256),Image.ANTIALIAS)
                img = ImageTk.PhotoImage(img)
                imgs.append(img) # Imageをリストに追加
                canvas = tk.Canvas(bg = "purple", width=256, height=256)
                canvas.place(x=(j+2)*image_padding, y=i*image_padding) 
                canvas.create_image(3, 3, image=img, anchor=tk.NW,tag=str(i*Y+j))
                canvases.append(canvas)
                # ボタンの設置
                button = tk.Button(root, text=str(i*Y+j), width=3, height=1, bg="#E6E6E6")
                button.place(x=(j+2.35)*image_padding,y=(i+0.85)*image_padding)
                button.config(command=click_button(i*Y+j,button))
                buttons.append(button)
        evolution_label = tk.Label(text=evolve_explanation, background='white', foreground='black',font=("Lucida Console","15"))
        evolution_label.place(x=150, y=400)
        labels.append(evolution_label)
        button = tk.Button(root,text = "evolve", bg="orange",command=click_ok_button)
        button.place(x=250,y=max(2.65,(X-0.35))*image_padding)
        root.mainloop()

100%|██████████| 50/50 [00:05<00:00,  9.14it/s]
/tmp/ipykernel_2982758/419107571.py:108: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((256,256),Image.ANTIALIAS)
/tmp/ipykernel_2982758/419107571.py:120: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((256,256),Image.ANTIALIAS)
100%|██████████| 50/50 [00:05<00:00,  9.24it/s]
/tmp/ipykernel_2982758/419107571.py:77: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((256,256),Image.ANTIALIAS)


In [9]:
import shutil